In [1]:
import os
import yaml

config = yaml.safe_load(open("myconfig.yml"))

os.environ["OPENAI_API_KEY"] = config["OPEN_AI_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

# Tools

### Memory as a retrieval tool

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

In [6]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

### Langchain tools

More tools here https://python.langchain.com/docs/integrations/tools

In [9]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

In [8]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [11]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [13]:
# run the agent
agent_chain.run(
    "What are the Neurons Lab services",
)



> Entering new AgentExecutor chain...
Thought: I should use the Neurons Lab search tool to find information on the services they offer.
Action:
```
{
  "action": "neurons_lab_search",
  "action_input": "services"
}
```
Observation: Grow your business & attract clients 
Do you also seek development opportunities or funding? Thanks to our advanced-tier AWS partnership and access to a vast VC network, we can aid your company in obtaining grants and other assistance.

















Talk to our expert team from Europe
Get in touch








Talk to our expert team from USA
Get in touch





Articles = mass of insights /
volume of experience

ALL POSTINGS

                                    1 — 6                                    











AI for portfolio management: from Markowitz to Reinforcement Learning

                                                    Jul 16, 2023|11 min read                                                


FinTech









Accelerate Remote Patient Monitorin

'The Neurons Lab offers services in AI feasibility analysis, AI solution engineering, AI operations management, and Generative AI Solutions. They also have co-labs for collaboration and work with companies in industries such as HealthTech, CleanTech, and RetailTech. You can get in touch with them to explore further.'

### Custom tools and reasoning

Here is a full guide: https://python.langchain.com/docs/modules/agents/tools/custom_tools

Reasoning: https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html

In [15]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [18]:
@tool
def calculate_length_tool(a: str) -> int:
    """Calculare the length of the text line"""
    return len(a)

In [23]:
calculate_length_tool.description, calculate_length_tool.args,

('calculate_length_tool(a: str) -> int - Calculare the length of the text line',
 {'a': {'title': 'A', 'type': 'string'}})

In [33]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool, calculate_length_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [34]:
# run the agent
agent_chain.run(
    "What is the length of the description of Neurons Lab services",
)



> Entering new AgentExecutor chain...
I should use the calculate_length_tool to find the length of the description of Neurons Lab services.
Action: calculate_length_tool
Action Input: Neurons Lab services description
Observation: 32
Thought:I need to search for the description of Neurons Lab services.
Action: neurons_lab_search
Action Input: Neurons Lab services description
Observation: Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech





AWS Certified
advanced partner






AI2 = Accelerators2 + 
Handbook2


AI feasibility analysis
Discover


AI solution engineering
Launch


AI operations management
S

'The length of the description of Neurons Lab services is 32.'